# Python - OpenCV

介绍和深度学习数据处理阶段最相关的基础使用，并完成4个有趣实用的小例子：

* 延时摄影小程序
* 视频中截屏采样的小程序
* 图片数据增加（data augmentation）的小工具
* 物体检测框标注小工具

## OpenCV 简介
OpenCV是计算机视觉领域应用最广泛的开源工具包，基于C/C++，支持Linux/Windows/MacOS/Android/iOS，并提供了Python，Matlab和Java等语言的接口.
### OpenCV的结构
>和Python一样，当前的OpenCV也有两个大版本，OpenCV2和OpenCV3。相比OpenCV2，OpenCV3提供了更强的功能和更多方便的特性。不过考虑到和深度学习框架的兼容性，以及上手安装的难度，这部分先以2为主进行介绍。从使用的角度来看，和OpenCV2相比，OpenCV3的主要变化是更多的功能和更细化的模块划分。

根据功能和需求的不同，OpenCV中的函数接口大体可以分为如下部分：

* core：核心模块，主要包含了OpenCV中最基本的结构（矩阵，点线和形状等），以及相关的基础运算/操作。
* imgproc：图像处理模块，包含和图像相关的基础功能（滤波，梯度，改变大小等），以及一些衍生的高级功能（图像分割，直方图，形态分析和边缘/直线提取等). 
* highgui：提供了用户界面和文件读取的基本函数，比如图像显示窗口的生成和控制，图像/视频文件的IO等。

如果不考虑视频应用，以上三个就是最核心和常用的模块了. 

针对**视频和一些特别的视觉应用**，OpenCV也提供了强劲的支持：
* video：用于视频分析的常用功能，比如光流法（Optical Flow）和目标跟踪等。
* calib3d：三维重建，立体视觉和相机标定等的相关功能。
* features2d：二维特征相关的功能，主要是一些不受专利保护的，商业友好的特征点检测和匹配等功能，比如ORB特征。
* object：目标检测模块，包含级联分类和Latent SVM
* ml：机器学习算法模块，包含一些视觉中最常用的传统机器学习算法。
* flann：最近邻算法库，Fast Library for Approximate Nearest Neighbors，用于在多维空间进行聚类和检索，经常和关键点匹配搭配使用。
* gpu：包含了一些gpu加速的接口，底层的加速是CUDA实现。
* photo：计算摄像学（Computational Photography）相关的接口，当然这只是个名字，其实只有图像修复和降噪而已。
* stitching：图像拼接模块，有了它可以自己生成全景照片。
* nonfree：受到专利保护的一些算法，其实就是SIFT和SURF。
* contrib：一些实验性质的算法，考虑在未来版本中加入的。
* legacy：字面是遗产，意思就是废弃的一些接口，保留是考虑到向下兼容。
* ocl：利用OpenCL并行加速的一些接口。
* superres：超分辨率模块，其实就是BTV-L1（Biliteral Total Variation – L1 regularization）算法
* viz：基础的3D渲染模块，其实底层就是著名的3D工具包VTK（Visualization Toolkit）。

#### 基本使用
图像就是一个矩阵，在OpenCV for Python中，图像就是NumPy中的数组, 图像使用NumPy数组的属性来表示图像的尺寸和通道信息

##### 读取图像

读图像用`cv2.imread()`，可以按照不同模式读取，一般最常用到的是读取单通道灰度图，或者直接默认读取多通道。存图像用`cv2.imwrite()`，注意存的时候是没有单通道这一说的, 根据保存文件名的后缀和当前的array维度，OpenCV自动判断存的通道，另外压缩格式还可以指定存储质量.


In [ ]:
# -----------------------------
# 读取图像
# -----------------------------

import cv2

img = cv2.imread('little_white_dog.jpeg')
cv2.namedWindow('Image')
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(img.shape)



In [ ]:
import cv2
import numpy as np

img = cv2.imread('figure.jpg')
px = img[100,100] 
# [128 122 153]
print( px)  
blue = img[100,100,0]  
print(blue)

# 以上这种读取像素值的方式非常缓慢。推荐使用Numpy的函数——array.item()和array.itemset()来访问
# accessing red value  
red = img.item(100, 100, 2)
print(red)

# modifying RED value  
img.itemset((100,100,2), 10)
img.item(100, 100, 2)

# 获取图像的属性——行数、列数、通道数、图像的数据类型以及像素点的数量等
img.shape
# 获得像素点的数量
img.size
img.dtype

In [ ]:
import cv2
import numpy as np

color_img = cv2.imread('test_400x600.jpeg')
print(color_img.shape)

# 直接读取单通道
gray_img = cv2.imread('test_400x600.jpeg', cv2.IMREAD_GRAYSCALE)
print(gray_img.shape)

# 把单通道图片保存后，再读取，仍然是3通道，相当于把单通道值复制到3个通道保存
cv2.imwrite('test_grayscale.jpeg', gray_img)
reload_grascale = cv2.imread('test_grayscale.jpeg')
print(reload_grascale.shape)

# cv2.IMWRITE_JPEG_QUALITY指定jpg质量，范围0到100，默认95，越高画质越好，文件越大
cv2.imwrite('test_imwrite.jpeg', color_img, (int(cv2.IMWRITE_JPEG_QUALITY), 80))

cv2.imwrite('test_imwrite.png', color_img, (int(cv2.IMWRITE_PNG_COMPRESSION), 5))

##### 创建/复制图像

In [ ]:
# -----------------------------
# 创建/复制图像
# 图像就是一个矩阵，在OpenCV for Python中，图像就是NumPy中的数组
# 如果要创建图像，需要使用numpy的函数, 图像使用NumPy数组的属性来表示图像的尺寸和通道信息
# -----------------------------
import cv2
import numpy as np

img = cv2.imread('little_white_dog.jpeg')
emptyImage = np.zeros(img.shape, np.uint8)

emptyImage2 = img.copy()

emptyImage3 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# emptyImage3[...] = 0

cv2.imshow('EmptyImage', emptyImage)
cv2.imshow('Image', img)
cv2.imshow('EmptyImage2', emptyImage2)
cv2.imshow('EmptyImage3', emptyImage3)

# 第三个参数针对特定的格式： 对于JPEG，其表示的是图像的质量，
# 用0-100的整数表示，默认为95。 注意，cv2.IMWRITE_JPEG_QUALITY类型为Long，必须转换成int。下面是以不同质量存储的两幅图：

cv2.imwrite('little_white_dog2.jpg', img, [int(cv2.IMWRITE_JPEG_QUALITY), 5])
cv2.imwrite('little_white_dog3.jpg', img, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

# 对于PNG，第三个参数表示的是压缩级别。cv2.IMWRITE_PNG_COMPRESSION，从0到9,压缩级别越高，图像尺寸越小。默认级别为3：

cv2.imwrite('little_white_dog2.png', img, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
cv2.imwrite('little_white_dog3.png', img, [int(cv2.IMWRITE_PNG_COMPRESSION), 9])

cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# numpy 全复制  不是镜像
import cv2
import numpy as np

img = cv2.imread('little_white_dog.jpeg')
c = np.zeros(img.shape, dtype=img.dtype)
c[:, :, :] = img[:, :, :]
c is img  # false
c.base is img  # false

##### 图像元素的访问
**与C++不同，在Python中灰度图的img.ndim = 2，而C++中灰度图图像的通道数img.channel() =1**

这里使用了numpy的随机数，Python自身也有一个随机数生成函数。这里只是一种习惯，np.random模块中拥有更多的方法，而Python自带的random只是一个轻量级的模块。不过需要注意的是np.random.seed()不是线程安全的，而**Python自带的random.seed()是线程安全**的。如果使用随机数时需要用到多线程，建议使用Python自带的random()和random.seed()，或者构建一个本地的np.random.Random类的实例。


In [ ]:
# -----------------------------
# 图像元素的访问
# 像素的访问和访问numpy中ndarray的方法完全一样
# 下面通过对图像添加人工的椒盐现象来进一步说明OpenCV Python中需要注意的一些问题
# -----------------------------

import cv2
import numpy as np

def salt(img, n):
    for k in range(n):
        j = int(np.random.random() * img.shape[0])
        i = int(np.random.random() * img.shape[1])
        if img.ndim == 2:
            img[j, i] = 255
        elif img.ndim ==3:
            img[j, i, 0] = 255
            img[j, i, 1] = 255
            img[j, i, 2] = 255
    return img
        
if __name__ == '__main__':
    img = cv2.imread('iPhone.png')
    saltImage = salt(img, 500)
    cv2.imshow('Salt', saltImage)
    cv2.waitKey(0)
    cv2.destoryAllWindows()
    

##### 缩放，裁剪和补边
缩放通过`cv2.resize()`实现(**指定大小的格式是(宽度,高度)**)，裁剪则是利用array自身的下标截取实现，此外OpenCV还可以给图像补边，这样能对一幅图像的形状和感兴趣区域实现各种操作。

下面的例子中读取一幅400×600分辨率的图片，并执行一些基础的操作：

In [ ]:


# 读取一张600x375分辨率的图像
img = cv2.imread('figure.jpg')
# 缩放成200x200的方形图像
img_200x200 = cv2.resize(img, (200, 200))

# 不直接指定缩放后大小，通过fx和fy指定缩放比例，0.5则长宽都为原来一半
# 等效于img_200x300 = cv2.resize(img, (300, 200))，注意指定大小的格式是(宽度,高度)

# 插值方法默认是cv2.INTER_LINEAR，这里指定为最近邻插值
# fx、fy是沿x轴和y轴的缩放系数 和 dsize不能同时为0
'''
最优一个参数interpolation表示插值方式，有以下几种：
INTER_NEAREST - 最近邻插值
INTER_LINEAR - 线性插值（默认）
INTER_AREA - 区域插值
INTER_CUBIC - 三次样条插值
INTER_LANCZOS4 - Lanczos插值
'''
img_200x300 = cv2.resize(img, (200, 300), fx=0.5, fy=0.5, interpolation=cv2.INTER_NEAREST)

# 在上张图片的基础上，上下各贴50像素的黑边，生成300x300的图像
img_300x300 = cv2.copyMakeBorder(img, 50, 50, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))

# 对照片中人脸的部分进行剪裁  高度区间和宽度区间  
# patch_tree = img[20:150, -300:-10]
patch_tree = img[7:150, 300:450]

cv2.imwrite('cropped_tree.jpg', patch_tree)
cv2.imwrite('resize_200x300.jpg', img_200x300)
cv2.imwrite('bordered_300x300.jpg', img_300x300)
cv2.imwrite('resize_200x200.jpg', img_200x200)


##### 色调，明暗，直方图和Gamma曲线

除了区域，图像本身的属性操作也非常多，比如可以通过HSV空间对色调和明暗进行调节。(HSV分别是色调(Hue), 饱和度（Saturation）和明度（Value）。在HSV空间中进行调节就避免了直接在RGB空间中调节是还需要考虑三个通道的相关性. OpenCV中H的取值是[0, 180)，其他两个通道的取值都是[0, 256). 

下面例子接着上面例子代码，通过HSV空间对图像进行调整：

In [1]:
import cv2

img = cv2.imread('figure.jpg')

# 通过cv2.cvtColor把图像从BGR转换到HSV
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

turn_green = img_hsv.copy()
colorless_hsv = img_hsv.copy()
darker_hsv = img_hsv.copy()

# H空间中，绿色比黄色的值高一点，所以给每个像素+15，黄色的树叶就会变绿
turn_green[:, :, 0] = (turn_green[:, :, 0]+15) % 180
turn_green_img = cv2.cvtColor(turn_green, cv2.COLOR_HSV2BGR)
cv2.imwrite('trun_green.jpg', turn_green_img)

# 减小饱和度会让图像损失鲜艳，变得更灰
colorless_hsv[:, :, 1] = 0.5 * colorless_hsv[:, :, 1]
colorless_img = cv2.cvtColor(colorless_hsv, cv2.COLOR_HSV2BGR)
cv2.imwrite('colorless.jpg', colorless_img)

# 减小明度为原来一半
darker_hsv[:, :, 2] = 0.5 * darker_hsv[:, :, 2]
darker_img = cv2.cvtColor(darker_hsv, cv2.COLOR_HSV2BGR)
cv2.imwrite('darker.jpg', darker_img)

True

**直方图**

无论是HSV还是RGB，我们都较难一眼就对像素中值的分布有细致的了解，这时候就需要直方图。如果直方图中的成分过于靠近0或者255，可能就出现了**暗部细节不足或者亮部细节丢失**的情况。

这个时候，一个常用方法是考虑用Gamma变换来提升暗部细节。Gamma变换是矫正相机直接成像和人眼感受图像差别的一种常用手段，

简单来说就是通过非线性变换让图像从对曝光强度的线性响应变得更接近人眼感受到的响应。


In [15]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

'''
cv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate ]]) 返回hist
其中第一个参数必须用方括号括起来。
第二个参数是用于计算直方图的通道
第三个参数是Mask，这里没有使用，所以用None。
第四个参数是histSize，表示这个直方图分成多少份（即多少个直方柱）
第五个参数是表示直方图中各个像素的值, [0.0, 256.0]表示直方图能表示像素值从0.0到256的像素。
最后是两个可选参数，由于直方图作为函数结果返回了，所以第六个hist就没有意义了（待确定）
最后一个accumulate是一个布尔值，用来表示直方图是否叠加。
'''
img = cv2.imread('figure.jpg')

# 直方图 <class 'numpy.ndarray'> (256, 1)
hist_b = cv2.calcHist([img], [0], None, [256], [0, 256])
hist_g = cv2.calcHist([img], [1], None, [256], [0, 256])
hist_r = cv2.calcHist([img], [2], None, [256], [0, 256])

# 定义Gamma矫正的函数
def gamma_trans(img, gamma):
    # 具体做法是先归一化到1，然后gamma作为指数值求出新的像素值再还原
    gamma_table = [np.power(x/255.0, gamma)*255.0 for x in range(256)]
    gamma_table = np.round(np.array(gamma_table)).astype(np.uint8)
    
     # 实现这个映射用的是OpenCV的查找表函数
    return cv2.LUT(img, gamma_table)

# 执行Gamma矫正，小于1的值让暗部细节大量提升，同时亮部细节少量提升
img_corrected = gamma_trans(img, 0.5)
cv2.imwrite('img_corrected.jpg',img_corrected)

# 分通道计算Gamma矫正后的直方图
hist_b_corrected = cv2.calcHist([img_corrected], [0], None, [256], [0, 256])
hist_g_corrected = cv2.calcHist([img_corrected], [1], None, [256], [0, 256])
hist_r_corrected = cv2.calcHist([img_corrected], [2], None, [256], [0, 256])

fig = plt.figure()

pix_hists = [
    [hist_b, hist_g, hist_r],
    [hist_b_corrected, hist_g_corrected, hist_r_corrected]
]

pix_vals = range(256)
for sub_plt, pix_hist in zip([121, 122], pix_hists):
    ax = fig.add_subplot(sub_plt, projection='3d')
    for c, z, channel_hist in zip(['b', 'g', 'r'], [20, 10, 0], pix_hist):
        cs = [c] * 256
        ax.bar(pix_vals, channel_hist, zs=z, zdir='y', color=cs, alpha=0.618, edgecolor='none', lw=0)

    ax.set_xlabel('Pixel Values')
    ax.set_xlim([0, 256])
    ax.set_ylabel('Channels')
    ax.set_zlabel('Counts')

plt.show()
'''

fig = plt.figure(figsize=(8, 4))

pix_hists = [
    [hist_b, hist_g, hist_r],
    [hist_b_corrected, hist_g_corrected, hist_r_corrected]
]
pix_vals = range(256)

for sub_plt, pix_hist in zip([121, 122], pix_hists):
    ax = fig.add_subplot(sub_plt, projection='3d')
    for c, z, channel_hist in zip(['b', 'g', 'r'], [20, 10, 0], pix_hist):
        cs = [c] * 256
        ax.bar(pix_vals, channel_hist, zs=z, zdir='y', color=cs, alpha=0.618, edgecolor='none', lw=0)
        
    ax.set_xlabel('Pixel Values')
    ax.set_xlim([0, 256])
    ax.set_ylabel('Channels')
    ax.set_zlabel('Counts')
    
plt.show()
'''



TypeError: only length-1 arrays can be converted to Python scalars

##### 分离通道
由于OpenCV Python和NumPy结合的很紧, 所以即可以使用OpenCV自带的split函数，也可以直接操作numpy数组来分离通道。

In [ ]:
# -----------------------------
# 分离、合并通道
# 即可以使用OpenCV自带的split函数，也可以直接操作numpy数组来分离通道。
# 
# -----------------------------
import cv2

img = cv2.imread('little_white_dog.jpeg')
b, g, r = cv2.split(img)
cv2.imshow('Blue', b)
cv2.imshow('Green', g)
cv2.imshow('Red', r)

cv2.waitKey(0)
cv2.destroyAllWindows()  

# split返回RGB三个通道，如果只想返回其中一个通道, 最后的索引指出所需要的通道。
b2 = cv2.split(img)[0]

In [ ]:
# 也可以直接操作NumPy数组来达到这一目的：

import cv2
import numpy as np

img = cv2.imread('little_white_dog.jpeg')
b = np.zeros((img.shape[0], img.shape[1]), dtype=img.dtype)
g = np.zeros((img.shape[0], img.shape[1]), dtype=img.dtype)
r = np.zeros((img.shape[0], img.shape[1]), dtype=img.dtype)

b[:, :] = img[:, :, 0]
g[:, :] = img[:, :, 1]
r[:, :] = img[:, :, 2]

cv2.imshow('Blue', b)
cv2.imshow('green', g)
cv2.imshow('red', r)

cv2.waitKey(0)
cv2.destoryAllWindows()

##### 通道合并

**????????注意：这里只是演示，实际使用时请用OpenCV自带的merge函数！用NumPy组合的结果不能在OpenCV中其他函数使用，因为其组合方式与OpenCV自带的不一样，如下：**

In [ ]:
import cv2
import numpy as np

img = cv2.imread('little_white_dog.jpeg')
b = np.zeros((img.shape[0], img.shape[1]), dtype=img.dtype)
g = np.zeros((img.shape[0], img.shape[1]), dtype=img.dtype)
r = np.zeros((img.shape[0], img.shape[1]), dtype=img.dtype)

b[:, :] = img[:, :, 0]
g[:, :] = img[:, :, 1]
r[:, :] = img[:, :, 2]


# openCV 方法 merge
merged = cv2.merge([b, g, r])
print('merge by opencv')
# 在每个维数上以字节计算的步长
print(merged.strides)
# numpy 方法  dstack
mergedByNp = np.dstack([b, g, r])
print('merge by numpy')
print(mergedByNp.strides)


In [ ]:
# 关于stride
# 在每个维数上以字节计算的步长
import numpy as np

# a数组中每个元素都是NumPy中的整数类型，占8个字节，所以第一维中相邻元素之间的步长为8（个字节）。
a = np.arange(6)
# (8,)
print(a.strides)
print(a.dtype)  # int64

# 从里面开始看，里面是一个4个元素的一维整数数组，所以步长应该为8。外面是一个含有3个元素，每个元素的长度是8×4=32。所以步长为32。
b = np.arange(12).reshape(3, 4)
b.strides

# (160, 40, 8)
c = np.arange(60).reshape(3,4,5) # 三维, 行 , 列
c.strides

## Python-OpenCV基础

### 图像的表示
单通道的灰度图像在计算机中的表示，就是一个8位无符号整形的矩阵。

**在OpenCV中，默认的图像的表示确实反过来的，也就是BGR**, 比如在Python中，图像都是用numpy的array表示，但是同样的array在OpenCV中的显示效果和matplotlib中的显示效果就会不一样。

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

img = np.array([
    [[255, 0, 0], [0, 255, 0], [0, 0, 255]],
    [[255, 255, 0], [255, 0, 255], [0, 255, 255]],
    [[255, 255, 255], [128, 128, 128], [0, 0, 0]],
], dtype=np.uint8)

plt.figure('matplotlib & OpenCV', figsize=(8, 4))
# 用matplotlib存储
plt.imsave('img_pyplot.jpg', img)

# 用OpenCV存储
cv2.imwrite('img_cv2.jpg', img)


**不管是RGB还是BGR，都是高度×宽度×通道数，H×W×C的表达方式，而在深度学习中，因为要对不同通道应用卷积，所以用的是另一种方式：C×H×W，就是把每个通道都单独表达成一个二维矩阵.**
np.reshape() 😂 好像就是 CHW (不知道能否这么表述)
